In [42]:
#r "../src/NMoneys/bin/Debug/net6.0/NMoneys.dll"

using NMoneys;
using NMoneys.Support;
using System.Numerics;
using System.Reflection;
using System.Globalization;

record Diff(string Name, string FromCurrency, string FromGlobalization)
{
	private static string str<T>(T i) where T : IBinaryInteger<T> 
	{
		return i.ToString("D", CultureInfo.InvariantCulture);
	}
	private static string str<T>(T[] i) where T : IBinaryInteger<T> 
	{
		return string.Join('|', i);
	}

	public static Diff[] Build(Currency currency)
	{
		CanonicalCultureAttribute attr = currency.GetCanonicalCulture();
		if (attr is null) return Array.Empty<Diff>();

		CultureInfo ci = attr.Culture();
		NumberFormatInfo nfi = ci.NumberFormat;
		var ri = new RegionInfo(ci.Name);
		StringComparer ordinal = StringComparer.Ordinal;
		StringComparer native = StringComparer.Create(ci, false);

		List<Diff> diffs = new List<Diff>(10);
		if (currency.IsoCode != Currency.Code.Parse(ri.ISOCurrencySymbol))
		{
			diffs.Add(new Diff("ri.ISOCurrencySymbol", currency.AlphabeticCode, ri.ISOCurrencySymbol));
		}
		if (!ordinal.Equals(currency.EnglishName, ri.CurrencyEnglishName))
		{
			diffs.Add(new Diff("ri.CurrencyEnglishName", currency.EnglishName, ri.CurrencyEnglishName));
		}
		if (!native.Equals(currency.NativeName, ri.CurrencyNativeName))
		{
			diffs.Add(new Diff("ri.CurrencyNativeName", currency.NativeName, ri.CurrencyNativeName));
		}
		if (!native.Equals(currency.Symbol, ri.CurrencySymbol))
		{
			diffs.Add(new Diff("ri.CurrencySymbol", currency.Symbol, ri.CurrencySymbol));
		}
		if (currency.SignificantDecimalDigits != nfi.CurrencyDecimalDigits)
		{
			diffs.Add(new Diff("nfi.CurrencyDecimalDigits", str(currency.SignificantDecimalDigits), str(nfi.CurrencyDecimalDigits)));
		}
		if (!ordinal.Equals(currency.DecimalSeparator, nfi.CurrencyDecimalSeparator))
		{
			diffs.Add(new Diff("nfi.CurrencyDecimalSeparator", currency.DecimalSeparator, nfi.CurrencyDecimalSeparator));
		}
		if (!ordinal.Equals(currency.GroupSeparator, nfi.CurrencyGroupSeparator))
		{
			diffs.Add(new Diff("nfi.CurrencyGroupSeparator", currency.GroupSeparator, nfi.CurrencyGroupSeparator));
		}
		if (!currency.GroupSizes.SequenceEqual(nfi.CurrencyGroupSizes.Select(s => (byte)s)))
		{
			diffs.Add(new Diff("nfi.CurrencyGroupSizes", str(currency.GroupSizes), str(nfi.CurrencyGroupSizes)));
		}
		if (currency.PositivePattern != nfi.CurrencyPositivePattern)
		{
			diffs.Add(new Diff("nfi.CurrencyPositivePattern", str(currency.PositivePattern), str(nfi.CurrencyPositivePattern)));
		}
		if (currency.NegativePattern != nfi.CurrencyNegativePattern)
		{
			diffs.Add(new Diff("nfi.CurrencyNegativePattern", str(currency.NegativePattern), str(nfi.CurrencyNegativePattern)));
		}

		return diffs.ToArray();
	}
};

In [43]:
Currency.FindAll()
	.Select(c => new { Currency = c, Diffs = Diff.Build(c) })
	.Where(a => a.Diffs.Length  > 0)


[ 4, 4, 3, 5, 4, 3, 2, 1, 1, 5, 3, 1, 1, 5, 3, 1, 2, 2, 2, 4 ... (more) ]